# CS 4E03 Energy System Research Project 2

<b>Author</b>: Jingze Dai, McMaster University
<br>
<b>Supervisor</b>: Dr.Douglas Down, Department of Computing and Software, McMaster University

In [3]:
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from scipy.sparse.linalg import eigs
from scipy.linalg import solve
import time

# Count this program's running time
start_time = time.time()

def initialize_parameters():
    global arrival_rate, serve_rate, turnon_rate, idle_rate, idle_power_cost
    global k1, k2, will_turn_off
    global power_OFF_idle, power_OFF_on, power_ON_idle, power_ON_reg, power_ON_enhanced, beta
    
    """
    lambda - the arrival rate                                                       -> Constant
    mu     - the service rate                                                       -> Constant
    gamma  - the rate at which system turns on (turnon_rate)                        -> Constant
    alpha  - the rate at which system turns off with no tasks (idle_rate)           -> Constant
    sigma  - the power consumption in the idle state                                -> Constant
    """
    # To make the system stable, assume lambda < mu
    # The enhanced serve rate (lambda_bar) must be smaller than the normal serve rate
    # Both the turn on rate and the idle rate must be greater than 0
    arrival_rate = 0.05
    serve_rate = 1.0
    turnon_rate = 0.8
    idle_rate = 0.1
    idle_power_cost = 0.5

    """
    Threshold k1 - the amount of jobs triggering the energy system to turn on                          -> Variable
    Threshold k2 - the amount of jobs triggering the energy system to use the enhanced service rate    -> Variable
    The decision on whether the energy system will turn off once it becomes idle (True or False)       -> Variable
    """
    k1, k2 = 1, 5
    will_turn_off = False

    """
    Power consumption definitions
    Off states being idle (note serving any jobs): 0 power consumption               -> Constant
    Off states turning on: 1 power consumption                                       -> Constant
    On states being idle (note serving any jobs): 0.5 power consumption              -> Constant
    On states processing at regular rates: 1 power consumption                       -> Constant
    On states processing at enhanced rates: (enhanced rate) ^ 2 power consumption    -> Constant
    beta (Power consumption weights in the total cost calculation)                   -> Constant
    """
    power_OFF_idle = 0.0
    power_OFF_on = 1.0
    power_ON_idle = idle_power_cost
    power_ON_reg = serve_rate
    beta = 1

def setup_States():
    global max_jobs, states, n_states
    # States starting with "OFF" mean OFF, and states starting with "ON" mean ON
    # State name's number represents the amount of waiting jobs
    # e.g., State OFF_0 represents the system is off, and there is 0 job inside the system
    # Example: states = ['OFF_0', 'OFF_1', 'OFF_2', 'OFF_3', 'OFF_4', 'OFF_5', 'ON_0', 'ON_1', 'ON_2', 'ON_3', 'ON_4', 'ON_5']
    max_jobs = 500
    states = [f'OFF_{i}' for i in range(max_jobs + 1)] + [f'ON_{i}' for i in range(max_jobs + 1)]
    n_states = len(states)

# Exception Handling for discovering unexpected parameter values.
def exception_handling():
    # Part 1: Check whether constant parameters' values are in the appropriate range.
    if arrival_rate <= 0 or arrival_rate >= 1:
        raise Exception("Problem: Illegal arrival rate values")
    if serve_rate != 1:
        raise Exception("Problem: Illegal service rate values")
    if arrival_rate >= serve_rate:
        raise Exception("Problem: Unstable System: arrival rate > serving rate")
    if turnon_rate <= 0:
        raise Exception("Problem: Illegal turn-on rate values")
    if idle_rate <= 0:
        raise Exception("Problem: Illegal idle rate values")
    if idle_power_cost <= 0 or idle_power_cost >= 1:
        raise Exception("Problem: Illegal idle power cost values")    
    # Part 2: Check whether variable parameters' values are in the appropriate range.
    if k1 > k2:
        raise Exception("Problem: Illegal threshold k1 and k2 values")
    if not (1 <= k1 <= max_jobs):
        raise Exception("Problem: Illegal threshold k1 value")
    if not (1 <= k2 <= max_jobs):
        raise Exception("Problem: Illegal threshold k2 value")


# Construct the transition matrix for further computing
def setup_Transition_Matrix():
    global transition_matrix, transitions, state_indices, transition_df
    transition_matrix = np.zeros((n_states, n_states))
    transitions = {}

    # Adding transitions for all ON and OFF states
    for i in range(max_jobs):
        transitions[(f'OFF_{i}', f'OFF_{i + 1}')] = arrival_rate
        transitions[(f'ON_{i}', f'ON_{i + 1}')] = arrival_rate

    for i in range(k1, max_jobs + 1):
        transitions[(f'OFF_{i}', f'ON_{i}')] = turnon_rate

    for i in range(1, max_jobs + 1):
        transitions[(f'ON_{i}', f'ON_{i - 1}')] = serve_rate
    
    # The idle rate only works when the system determines to turn off
    # Otherwise, the system will not go back to state 'OFF_0'
    if will_turn_off is True:
        transitions[('ON_0', 'OFF_0')] = idle_rate
    
    state_indices = {state: i for i, state in enumerate(states)}
    for (from_state, to_state), rate in transitions.items():
        i, j = state_indices[from_state], state_indices[to_state]
        transition_matrix[i, j] = rate

    row_sums = transition_matrix.sum(axis=1)
    for i in range(n_states):
        transition_matrix[i, i] = -row_sums[i]

    # Convert to CSR format (or keep as a dense array if needed for other operations)
    transition_matrix = csr_matrix(transition_matrix)

    # Convert the sparse matrix to a dense array for DataFrame construction
    transition_df = pd.DataFrame(transition_matrix.toarray(), index=states, columns=states)

# Calculate each states' steady-state probability
def calculate_Steady_State(probabilityPrint=False, errorPrint=True):
    global steady_state_probs, steady_state_df, min_prob

    Q = transition_matrix.toarray().T
    Q[-1, :] = 1  # Replace the last equation with the normalization condition

    b = np.zeros(n_states)
    b[-1] = 1

    steady_state_probs = solve(Q, b)
    steady_state_df = pd.DataFrame(steady_state_probs, index=states, columns=["Probability"])

    # Checkes whether or not there are null and indefinite values 
    # Because null and infinite values are serious problems, we have to shut down running
    if np.any(np.isnan(steady_state_probs)) or np.any(np.isinf(steady_state_probs)):
        print("Error: Steady-state probabilities contain NaNs or infinities.")
        return

    # Print steady-state proability if required
    if probabilityPrint:
        print("Steady-State Probabilities:")
        print(steady_state_df)

    # Check the magnitude of negative steady-state probability values
    # If all negative values are too small, then it is OK
    min_prob = steady_state_df.min().values[0]
    if abs(min_prob) >= 1e-8 and min_prob < 0:
        if errorPrint:
            print(f"The minimum steady-state probability {min_prob} is significant.")
    elif errorPrint:
        print(f"The minimum steady-state probability {min_prob} approaches 0, can be ignored.")

    # Check the amount of negative steady-state probability values
    negative_indices = np.where(steady_state_probs < 0)[0]
    if len(negative_indices) > 0:
        if errorPrint:
            print(f"Warning: Found {len(negative_indices)} negative values in steady-state probabilities")
        # Optionally handle negative values here
    elif errorPrint:
        print("NO NEGATIVE steady-state probability values")

    # Check whether all steady-state probability values sum up to 1.
    sum_prob = steady_state_probs.sum()
    if not np.isclose(sum_prob, 1):
        if errorPrint:
            print(f"Warning: The sum of steady-state probabilities is {sum_prob}, which is not close to 1.")
    else:
        if errorPrint:
            print("Sum is 1, Good.")

# Power consumption calculations
# Example: State A has steady state probability 0.3, State B 0.5, and State C 0.2.
# State A power consumption 1, State B 2, and State C 3.
# Total energy consumption = 1 * 0.3 + 2 * 0.5 + 3 * 0.2 = 1.9. -> Suming up each state's power consumption.
def calculate_Power_Consumption():
    global power_consumption, power_consumption_sum, power_consumption_list
    power_consumption = {f'OFF_{i}': power_OFF_on if i >= k1 else power_OFF_idle for i in range(max_jobs + 1)}
    power_consumption.update({f'ON_{i}': power_ON_reg for i in range(max_jobs + 1)})
    power_consumption.update({'ON_0': power_ON_idle})

    # Compute each state's power consumption, and recording these values on a list
    power_consumption_list = np.array(
        [steady_state_probs[i] * power_consumption[states[i]] for i in range(len(steady_state_probs))])
    power_consumption_sum = np.sum(power_consumption_list) # Calculate the whole system's energy consumption
    
# Calculate Total Cost
def calculate_Cost(steady_state_df):
    global total_cost, expected_jobs
    probabilities = np.array(steady_state_df["Probability"].values)
    jobs = np.array([int(state.split('_')[1]) for state in steady_state_df.index])
    expected_jobs = np.dot(jobs, probabilities)

    # Formula: Total cost = total power consumption * the power consumption weights + expected number of jobs
    total_cost = power_consumption_sum * beta + expected_jobs

# Model Execution
initialize_parameters()
setup_States()
exception_handling()
setup_Transition_Matrix()
calculate_Steady_State(False, True)
calculate_Power_Consumption()
calculate_Cost(steady_state_df)

# Display various results below
#pd.set_option('display.max_rows', None)
#pd.set_option('display.max_columns', None)
#pd.set_option('display.max_colwidth', None)

print("\nTransition Matrix:")
print(transition_df)
print("\n")

print("\nSteady-State Probabilities:")
print(steady_state_df)
print("\n")

# Print out first 10 OFF states
print(steady_state_df.loc['OFF_0'])
print(steady_state_df.loc['OFF_1'])
print(steady_state_df.loc['OFF_2'])
print(steady_state_df.loc['OFF_3'])
print(steady_state_df.loc['OFF_4'])
print(steady_state_df.loc['OFF_5'])
print(steady_state_df.loc['OFF_6'])
print(steady_state_df.loc['OFF_7'])
print(steady_state_df.loc['OFF_8'])
print(steady_state_df.loc['OFF_9'])
print("\n")

# Print out first 10 ON states
print(steady_state_df.loc['ON_0'])
print(steady_state_df.loc['ON_1'])
print(steady_state_df.loc['ON_2'])
print(steady_state_df.loc['ON_3'])
print(steady_state_df.loc['ON_4'])
print(steady_state_df.loc['ON_5'])
print(steady_state_df.loc['ON_6'])
print(steady_state_df.loc['ON_7'])
print(steady_state_df.loc['ON_8'])
print(steady_state_df.loc['ON_9'])
print("\n")

print("\nIndividual-State Expected Power Consumption Values:")
print(power_consumption)

print("\nEach State's Actual Power Consumption (During Calculation):")
print(power_consumption_list)

print("\nTotal Power Consumption: " + str(power_consumption_sum))

print("\nTotal Cost: " + str(total_cost))

end_time = time.time()
elapsed_time = end_time - start_time
print(f"\nCalculation time: {elapsed_time:.6f} seconds")

The minimum steady-state probability -7.87320649839861e-18 approaches 0, can be ignored.
Sum is 1, Good.

Transition Matrix:
        OFF_0  OFF_1  OFF_2  OFF_3  OFF_4  OFF_5  OFF_6  OFF_7  OFF_8  OFF_9  \
OFF_0   -0.05   0.05   0.00   0.00   0.00   0.00    0.0    0.0    0.0    0.0   
OFF_1    0.00  -0.85   0.05   0.00   0.00   0.00    0.0    0.0    0.0    0.0   
OFF_2    0.00   0.00  -0.85   0.05   0.00   0.00    0.0    0.0    0.0    0.0   
OFF_3    0.00   0.00   0.00  -0.85   0.05   0.00    0.0    0.0    0.0    0.0   
OFF_4    0.00   0.00   0.00   0.00  -0.85   0.05    0.0    0.0    0.0    0.0   
...       ...    ...    ...    ...    ...    ...    ...    ...    ...    ...   
ON_496   0.00   0.00   0.00   0.00   0.00   0.00    0.0    0.0    0.0    0.0   
ON_497   0.00   0.00   0.00   0.00   0.00   0.00    0.0    0.0    0.0    0.0   
ON_498   0.00   0.00   0.00   0.00   0.00   0.00    0.0    0.0    0.0    0.0   
ON_499   0.00   0.00   0.00   0.00   0.00   0.00    0.0    0.0    0.0    0.